Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.


In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

Fiter the relevant links using GPT4o mini

In [4]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "You should respond in JSON as in this example:"
    user_prompt += """
    {
        "links": [
            {"type": "about page", "url": "https://full.url/goes/here/about"},
            {"type": "careers page", "url": "https://another.full.url/careers"},
            {"type": "contact page", "url": "https://full.url/goes/here/contact"},
            {"type": "services page", "url": "https://full.url/goes/here/services"},
            {"type": "blog page", "url": "https://full.url/goes/here/blog"}
        ]
    }"""
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [6]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [7]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 'blog/inference-providers-nebius-novita-hyperbolic',
 '/spaces',
 '/models',
 '/perplexity-ai/r1-1776',
 '/deepseek-ai/DeepSeek-R1',
 '/microsoft/OmniParser-v2.0',
 '/stepfun-ai/Step-Audio-Chat',
 '/Zyphra/Zonos-v0.1-hybrid',
 '/models',
 '/spaces/nanotron/ultrascale-playbook',
 '/spaces/lllyasviel/LuminaBrush',
 '/spaces/black-forest-labs/FLUX.1-dev',
 '/spaces/m-ric/open_Deep-Research',
 '/spaces/microsoft/OmniParser-v2',
 '/spaces',
 '/datasets/Congliu/Chinese-DeepSeek-R1-Distill-data-110k',
 '/datasets/open-r1/OpenR1-Math-220k',
 '/datasets/open-thoughts/OpenThoughts-114k',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/facebook/natural_reasoning',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook

In [8]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'docs page', 'url': 'https://huggingface.co/docs'}]}

### Creating brocher using LLM

In [9]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [10]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'contact page', 'url': 'https://discuss.huggingface.co'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Welcome Hyperbolic, Nebius AI Studio, and Novita on the Hub 🔥
Welcome Fireworks.ai on the Hub 🎆
Welcome to Inference Providers on the Hub 🔥
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
perplexity-ai/r1-1776
Updated
about 23 hours ago
•
4.36k
•
1.12k
deepseek-ai/DeepSeek-R1
Updated
11 days ago
•
4.32M
•
9.74k
microsoft/OmniParser-v2.0
Updated
3 days ago
•

In [11]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information.\
Example:\n\
# Company Brochure\n\
\n\
## About Us\n\
[Provide a brief description of the company, its mission, and values.]\n\
\n\
## Our Culture\n\
[Describe the company culture, work environment, and core values.]\n\
\n\
## Our Customers\n\
[Highlight key customers, industries served, and customer testimonials.]\n\
\n\
## Careers\n\
[Provide information about career opportunities, benefits, and how to apply.]\n\
\n\
## Contact Us\n\
[Include contact information, office locations, and social media links.]\n\
"


In [12]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [13]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nWelcome Hyperbolic, Nebius AI Studio, and Novita on the Hub 🔥\nWelcome Fireworks.ai on the Hub 🎆\nWelcome to Inference Providers on the Hub 🔥\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nperplexity-ai/r1-1776\nUpdated\nabout 23 hours ago\n•\n4.36k\n•\n1.12k\ndeepseek-ai/DeepSeek-R1\nUpdated\n11 days ago\n•\n4.32M\n•\n9.74k\nmicrosoft/OmniParser-v2.0\nUpdated\n3 days ago\n•\n3.18k\n•\n729\nstepfun-ai/Step-Audio-Chat\nUpdated\n3 days

In [20]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))
    return result

In [21]:
brocher =create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'contact page', 'url': 'https://huggingface.co/contact'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


# Company Brochure: Hugging Face

## About Us
Hugging Face is a pioneering AI community that is building the future of machine learning through collaboration and innovation. We serve as a comprehensive platform for the machine learning community, offering a space to explore, create, and share models, datasets, and applications. Our mission is to democratize AI by providing the tools and resources that enable anyone to engage with machine learning technology.

## Our Culture
At Hugging Face, we foster a collaborative and inclusive culture that encourages creativity and innovation. Our core values revolve around community engagement, transparency, and open-source development, ensuring that everyone feels empowered to contribute. We believe in creating an environment that supports professional growth, shared learning, and a passion for cutting-edge technology.

## Our Customers
Hugging Face supports over 50,000 organizations, including industry leaders such as Amazon Web Services, Google, Microsoft, and Grammarly. We serve a diverse range of sectors from enterprise companies to non-profits, helping them harness the power of machine learning for their specific needs. Our user base is deeply engaged, with many users contributing to our growing collection of over 1 million models and datasets.

### Customer Testimonials
Our community praises our platform for its user-friendliness and extensive resources, stating that collaboration on Hugging Face significantly accelerates their machine learning projects.

## Careers
We are always on the lookout for passionate individuals to join our team. At Hugging Face, we offer rewarding career opportunities across various roles in technology and business. We value a diverse workforce and provide competitive benefits, including flexible working hours, remote work options, and professional development programs. 

### Current Openings
Explore our career page to find out more about our current job openings and how to apply.

## Contact Us
For further inquiries, feel free to reach out:
- **Website:** [HuggingFace.com](https://huggingface.co)
- **Social Media:** 
  - [Twitter](https://twitter.com/huggingface)
  - [LinkedIn](https://linkedin.com/company/huggingface)
  - [GitHub](https://github.com/huggingface)
  - [Discord](https://discord.gg/huggingface)

Join us at Hugging Face, where innovation and collaboration drive the future of AI!

In [22]:
def translate_to_hindi(text):
    translation_prompt = "Translate the following brochure content to Hindi:\n\n" + text
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a translation assistant."},
            {"role": "user", "content": translation_prompt}
        ],
    )
    translated_result = response.choices[0].message.content
    display(Markdown(translated_result))
    return translated_result


In [23]:
hindi_brocher =translate_to_hindi(brocher)

# कंपनी ब्रोशर: हगिंग फेस

## हमारे बारे में
हगिंग फेस एक अग्रणी एआई समुदाय है जो सहयोग और नवाचार के माध्यम से मशीन लर्निंग का भविष्य बना रहा है। हम मशीन लर्निंग समुदाय के लिए एक समग्र मंच के रूप में कार्य करते हैं, जो मॉडल, डेटासेट और अनुप्रयोगों का अन्वेषण, निर्माण और साझा करने के लिए एक स्थान प्रदान करता है। हमारा मिशन एआई को लोकतांत्रिक बनाना है, जिससे हर कोई मशीन लर्निंग तकनीक से जुड़ने के लिए आवश्यक उपकरण और संसाधन प्राप्त कर सके।

## हमारी संस्कृति
हगिंग फेस में, हम एक सहयोगात्मक और समावेशी संस्कृति को बढ़ावा देते हैं जो रचनात्मकता और नवाचार को प्रेरित करती है। हमारे केंद्रीय मूल्य सामुदायिक सहभागिता, पारदर्शिता और ओपन-सोर्स विकास के चारों ओर घूमते हैं, यह सुनिश्चित करते हुए कि हर कोई योगदान देने के लिए सशक्त महसूस करे। हम एक ऐसा वातावरण बनाने में विश्वास करते हैं जो पेशेवर विकास, साझा सीखने और अत्याधुनिक तकनीक के प्रति जुनून का समर्थन करता है।

## हमारे ग्राहक
हगिंग फेस 50,000 से अधिक संगठनों का समर्थन करता है, जिसमें अमेज़न वेब सर्विसेज, गूगल, माइक्रोसॉफ्ट और ग्रामरली जैसे उद्योग के नेता शामिल हैं। हम विभिन्न क्षेत्रों की सेवाएँ प्रदान करते हैं, जैसे कि उद्यम कंपनियाँ और गैर-लाभकारी संगठनों, जिससे उन्हें अपनी विशेष आवश्यकताओं के लिए मशीन लर्निंग की शक्ति का उपयोग करने में मदद मिल सके। हमारा उपयोगकर्ता आधार गहराई से संलग्न है, जिसमें कई उपयोगकर्ता हमारे बढ़ते संग्रह में 1 मिलियन से अधिक मॉडल और डेटासेट में योगदान कर रहे हैं।

### ग्राहक प्रशंसापत्र
हमारा समुदाय हमारे प्लेटफॉर्म की उपयोगकर्ता-मित्रता और व्यापक संसाधनों की प्रशंसा करता है, यह कहते हुए कि हगिंग फेस पर सहयोग उनके मशीन लर्निंग प्रोजेक्ट्स को काफी गति देता है।

## करियर
हम हमेशा उत्साही व्यक्तियों की तलाश में रहते हैं जो हमारी टीम में शामिल हों। हगिंग फेस में, हम तकनीक और व्यवसाय में विभिन्न भूमिकाओं में पुरस्कृत करियर अवसर प्रदान करते हैं। हम एक विविध कार्यबल के महत्व को समझते हैं और प्रतिस्पर्धी लाभ प्रदान करते हैं, जिसमें लचीले कार्य घंटे, दूरस्थ कार्य विकल्प और पेशेवर विकास कार्यक्रम शामिल हैं।

### वर्तमान अवसर
हमारे करियर पृष्ठ पर जाकर हमारे वर्तमान नौकरी के अवसरों और आवेदन करने के तरीकों के बारे में अधिक जानें।

## हमसे संपर्क करें
अधिक पूछताछ के लिए, बेझिझक संपर्क करें:
- **वेबसाइट:** [HuggingFace.com](https://huggingface.co)
- **सोशल मीडिया:** 
  - [Twitter](https://twitter.com/huggingface)
  - [LinkedIn](https://linkedin.com/company/huggingface)
  - [GitHub](https://github.com/huggingface)
  - [Discord](https://discord.gg/huggingface)

हमसे जुड़ें हगिंग फेस पर, जहाँ नवाचार और सहयोग एआई के भविष्य को प्रेरित करते हैं!